# Spark in Action - Chapter 3 Scala Version - Lab 200

In [ ]:
import org.apache.spark.sql.functions.concat
import org.apache.spark.sql.functions.lit
import org.apache.spark.Partition
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Row
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.41:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1667056910713)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.concat
import org.apache.spark.sql.functions.lit
import org.apache.spark.Partition
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Row
import org.apache.spark.sql.SparkSession


In [ ]:
val spark = SparkSession
                .builder
                .appName("Restaurants in Wake County, NC")
                .master("local[*]")
                .getOrCreate

22/10/29 17:21:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4a8fea7d


In [ ]:
// Restaurants_in_Wake_County_NC.csv,
// stores it in a dataframe
var df = spark
            .read
            .format("csv")
            .option("header", "true")
            .load("../net.jgp.books.spark.ch03/data/Restaurants_in_Wake_County_NC.csv")

println("*** Right after ingestion")

*** Right after ingestion


df: org.apache.spark.sql.DataFrame = [OBJECTID: string, HSISID: string ... 13 more fields]


In [ ]:
df.show(5)
df.printSchema()
println("We have " + df.count + " records.")

+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|     HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER|  RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+--------+------------+-----------+-------------+
|    1001|04092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18T00:00:...|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|04092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08T00:00:...|       Food Stand|    6953|-78.78211173|35

In [ ]:
// Let's transform our dataframe
df =  df.withColumn("county", lit("Wake"))
        .withColumnRenamed("HSISID", "datasetId")
        .withColumnRenamed("NAME", "name")
        .withColumnRenamed("ADDRESS1", "address1")
        .withColumnRenamed("ADDRESS2", "address2")
        .withColumnRenamed("CITY", "city")
        .withColumnRenamed("STATE", "state")
        .withColumnRenamed("POSTALCODE", "zip")
        .withColumnRenamed("PHONENUMBER", "tel")
        .withColumnRenamed("RESTAURANTOPENDATE", "dateStart")
        .withColumnRenamed("FACILITYTYPE", "type")
        .withColumnRenamed("X", "geoX")
        .withColumnRenamed("Y", "geoY")
        .drop("OBJECTID","PERMITID","GEOCODESTATUS")

df: org.apache.spark.sql.DataFrame = [datasetId: string, name: string ... 11 more fields]


In [ ]:
df = df.withColumn("id",
      concat(df.col("state"), lit("_"), df.col("county"), lit("_"), df.col("datasetId")))

df: org.apache.spark.sql.DataFrame = [datasetId: string, name: string ... 12 more fields]


In [ ]:
// Shows at most 5 rows from the dataframe
println("*** Dataframe transformed")
df.show(5)

*** Dataframe transformed
+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+------------+-----------+------+-------------------+
|  datasetId|                name|            address1|address2|       city|state|       zip|           tel|           dateStart|             type|        geoX|       geoY|county|                 id|
+-----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+--------------------+-----------------+------------+-----------+------+-------------------+
|04092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18T00:00:...|       Restaurant|-78.66818477|35.78783803|  Wake|NC_Wake_04092016024|
|04092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08T00:00:...|       Food Stand|-78.78211173|35.73717591|  

In [ ]:
// for book only
val drop_cols=List("address2","zip","tel","dateStart",
              "geoX","geoY","address1","datasetId")
val dfUsedForBook = df.drop(drop_cols:_*)
dfUsedForBook.show(5, 15)
// end

+---------------+-----------+-----+---------------+------+---------------+
|           name|       city|state|           type|county|             id|
+---------------+-----------+-----+---------------+------+---------------+
|           WABA|    RALEIGH|   NC|     Restaurant|  Wake|NC_Wake_0409...|
|WALMART DELI...|       CARY|   NC|     Food Stand|  Wake|NC_Wake_0409...|
|CAROLINA SUS...|    RALEIGH|   NC|     Restaurant|  Wake|NC_Wake_0409...|
|THE CORNER V...|    RALEIGH|   NC|Mobile Food ...|  Wake|NC_Wake_0409...|
|   SUBWAY #3726|WAKE FOREST|   NC|     Restaurant|  Wake|NC_Wake_0409...|
+---------------+-----------+-----+---------------+------+---------------+
only showing top 5 rows



drop_cols: List[String] = List(address2, zip, tel, dateStart, geoX, geoY, address1, datasetId)
dfUsedForBook: org.apache.spark.sql.DataFrame = [name: string, city: string ... 4 more fields]


In [ ]:
df.printSchema()

root
 |-- datasetId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: string (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: string (nullable = true)
 |-- geoY: string (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)



In [ ]:
println("*** Looking at partitions")
val partitions = df.rdd.partitions
val partitionCount = partitions.length
println("Partition count before repartition: " + partitionCount)

*** Looking at partitions
Partition count before repartition: 1


partitions: Array[org.apache.spark.Partition] = Array(FilePartition(0,[Lorg.apache.spark.sql.execution.datasources.PartitionedFile;@5d3be711))
partitionCount: Int = 1


In [ ]:
df = df.repartition(4)
println("Partition count after repartition: " + df.rdd.partitions.length)

Partition count after repartition: 4


df: org.apache.spark.sql.DataFrame = [datasetId: string, name: string ... 12 more fields]


In [ ]:
spark.close()